In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [22]:
import sys,random,math
from collections import Counter
import numpy as np

f = open('gdrive/My Drive/Grokking/tasksv11/en/qa1_single-supporting-fact_train.txt','r')
raw = f.readlines()     #len(raw): 3000; raw[:3] ['1 Mary moved to the bathroom.\n', '2 John went to the hallway.\n', '3 Where is Mary? \tbathroom\t1\n']
f.close()

tokens = list()
for line in raw[0:1000]:
    tokens.append(line.lower().replace("\n","").split(" ")[1:])

print(len(tokens),tokens[0:3])

1000 [['mary', 'moved', 'to', 'the', 'bathroom.'], ['john', 'went', 'to', 'the', 'hallway.'], ['where', 'is', 'mary?', '\tbathroom\t1']]


In [23]:
vocab = set()
for sent in tokens:
    for word in sent:
        vocab.add(word)

vocab = list(vocab) #vocab len: 82
print("sent len:",len(sent),"; sent:",sent)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i  
    
def words2indices(sentence):
    idx = list()
    for word in sentence:
        idx.append(word2index[word])
    return idx

  
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

sent len: 5 ; sent: ['sandra', 'went', 'to', 'the', 'garden.']


In [0]:
np.random.seed(1)
embed_size = 10

# word embeddings
embed = (np.random.rand(len(vocab),embed_size) - 0.5) * 0.1           #embed.shape: (82, 10)

# embedding -> embedding (initially the identity matrix)
recurrent = np.eye(embed_size)                                        #recurrent.shape: (10, 10)

# sentence embedding for empty sentence
start = np.zeros(embed_size)

# embedding -> output weights
decoder = (np.random.rand(embed_size, len(vocab)) - 0.5) * 0.1        #decoder.shape: (10, 82)

# one hot lookups (for loss function)
one_hot = np.eye(len(vocab))

In [0]:
def predict(sent):
    
    layers = list()
    layer = {}
    layer['hidden'] = start
    layers.append(layer)

    loss = 0

    # forward propagate
    preds = list()
    for target_i in range(len(sent)):
        layer = {} 
        # try to predict the next term
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))    #layer['pred']).shape: (82,)

        loss += -np.log(layer['pred'][sent[target_i]])
                      
        # generate the next hidden state
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[sent[target_i]]  #layer['hidden'].shape: (10,)
        layers.append(layer)
        
    return layers, loss
  
#a = [10, 5,20, 81]
#predict(a)

In [53]:
# forward
for iter in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iter%len(tokens)][1:])   # eg tokens ['moved', 'to', 'the', 'bathroom.'] | eg sent [7, 21, 29, 34]
    layers,loss = predict(sent) 

    # back propagate
    for layer_idx in reversed(range(len(layers))):       # eg len(layers): 5       
        layer = layers[layer_idx]
        target = sent[layer_idx-1]

        if(layer_idx > 0):  # if not the first layer
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            # if the last layer - don't pull from a later one becasue it doesn't exist
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else: # if the first layer
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())



    # update weights
    start -= layers[0]['hidden_delta'] * alpha / float(len(sent))
    for layer_idx,layer in enumerate(layers[1:]):
        
        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta']) * alpha / float(len(sent))
        
        embed_idx = sent[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha / float(len(sent))
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / float(len(sent))
        
    if(iter % 1000 == 0):
        print("Perplexity:" + str(np.exp(loss/len(sent))))


Perplexity:82.17873506062428
Perplexity:82.02226933139343
Perplexity:81.89515309084378
Perplexity:81.7608584680317
Perplexity:81.5751884395345
Perplexity:81.26312475414024
Perplexity:80.66967250015341
Perplexity:79.42072058736662
Perplexity:76.38544996500065
Perplexity:66.24425744816017
Perplexity:32.892724125499505
Perplexity:21.224577857741803
Perplexity:19.145125610597926
Perplexity:17.59763813921389
Perplexity:15.453741629316943
Perplexity:12.368946607211964
Perplexity:9.152429960913144
Perplexity:7.262729889619236
Perplexity:6.235250493935901
Perplexity:5.574673900592536
Perplexity:5.1422952014435594
Perplexity:4.888386591690299
Perplexity:4.737186714527934
Perplexity:4.641349679783937
Perplexity:4.578618433071957
Perplexity:4.523228090459675
Perplexity:4.453715425442932
Perplexity:4.364217492381606
Perplexity:4.258190967825188
Perplexity:4.139872979689222


In [54]:
sent_index = 4

l,_ = predict(words2indices(tokens[sent_index]))

print(tokens[sent_index])

for i,each_layer in enumerate(l[1:-1]):
    input = tokens[sent_index][i]
    true = tokens[sent_index][i+1]
    pred = vocab[each_layer['pred'].argmax()]
    print("Prev Input:" + input + (' ' * (12 - len(input))) +\
          "True:" + true + (" " * (15 - len(true))) + "Pred:" + pred)

['sandra', 'moved', 'to', 'the', 'garden.']
Prev Input:sandra      True:moved          Pred:is
Prev Input:moved       True:to             Pred:to
Prev Input:to          True:the            Pred:the
Prev Input:the         True:garden.        Pred:bedroom.
